In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 #Install required packages
!pip install unidecode
!pip install word2number
!pip install datasets
!pip install transformers

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
import datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from functools import partial
import torch
from torch import nn
from transformers import BertTokenizerFast as BertTokenizer, BertModel, get_linear_schedule_with_warmup, AdamW

In [ ]:
pd.read_csv('/content/drive/MyDrive/544 project/data/jigsaw-toxic-comment-train.csv').shape

(223549, 8)

In [ ]:
#Load data
TRAIN_PATH = '/content/drive/MyDrive/544 project/data/train1.csv'
TRAIN_PATH2 = '/content/drive/MyDrive/544 project/data/train2.csv'
TRAIN_PATH3 = '/content/drive/MyDrive/544 project/data/train3.csv'
TEST_PATH = '/content/drive/MyDrive/544 project/data/test.csv'
comment_train1 = pd.read_csv(TRAIN_PATH)
comment_train2 = pd.read_csv(TRAIN_PATH2)
comment_train3 = pd.read_csv(TRAIN_PATH3)
comment_test = pd.read_csv(TEST_PATH)

In [ ]:
comment_train1 = comment_train1[['comment_text', 'toxic']]
comment_train2['toxic'] = comment_train2['Insult']
comment_train2 = comment_train2.drop(['Insult', 'Date'], axis=1)
comment_train3['toxic'] = comment_train3['class'].apply(lambda x: 1 if x in (1, 0) else 0)
comment_train3 = comment_train3[['comment_text', 'toxic']]
comment_train = pd.concat([comment_train1, comment_train2, comment_train3])
comment_train = comment_train.dropna().reset_index(drop=True)
comment_test = comment_test.drop(['id'], axis=1)

In [ ]:
print(
    'Training data shape:', comment_train.shape[0],
    '\n',
    'Testing data shape:', comment_test.shape[0]
 )

Training data shape: 252279 
 Testing data shape: 153164


# Data proprocessing

In [ ]:
def comment_clean(text):
    text = re.sub("\\n", " ", str(text))
    text = re.sub(r'[^a-zA-Z ]', '', text)
    #remove user_id and IP address
    text = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", '', text)
    #remove text staring with user
    text = re.sub("\[\[User.*",'', text)
    #remove url and links
    text = re.sub('^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', ' ', text)
    return text.strip()

def remove_extra_space(text):
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()

def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

contractions = { 
	"ain't": "am not / are not / is not / has not / have not",
	"aren't": "are not / am not",
	"can't": "cannot",
	"can't've": "cannot have",
	"'cause": "because",
	"could've": "could have",
	"couldn't": "could not",
	"couldn't've": "could not have",
	"didn't": "did not",
	"doesn't": "does not",
	"don't": "do not",
	"hadn't": "had not",
	"hadn't've": "had not have",
	"hasn't": "has not",
	"haven't": "have not",
	"he'd": "he had / he would",
	"he'd've": "he would have",
	"he'll": "he shall / he will",
	"he'll've": "he shall have / he will have",
	"he's": "he has / he is",
	"how'd": "how did",
	"how'd'y": "how do you",
	"how'll": "how will",
	"how's": "how has / how is / how does",
	"I'd": "I had / I would",
	"I'd've": "I would have",
	"I'll": "I shall / I will",
	"I'll've": "I shall have / I will have",
	"I'm": "I am",
	"I've": "I have",
	"isn't": "is not",
	"it'd": "it had / it would",
	"it'd've": "it would have",
	"it'll": "it shall / it will",
	"it'll've": "it shall have / it will have",
	"it's": "it has / it is",
	"let's": "let us",
	"ma'am": "madam",
	"mayn't": "may not",
	"might've": "might have",
	"mightn't": "might not",
	"mightn't've": "might not have",
	"must've": "must have",
	"mustn't": "must not",
	"mustn't've": "must not have",
	"needn't": "need not",
	"needn't've": "need not have",
	"o'clock": "of the clock",
	"oughtn't": "ought not",
	"oughtn't've": "ought not have",
	"shan't": "shall not",
	"sha'n't": "shall not",
	"shan't've": "shall not have",
	"she'd": "she had / she would",
	"she'd've": "she would have",
	"she'll": "she shall / she will",
	"she'll've": "she shall have / she will have",
	"she's": "she has / she is",
	"should've": "should have",
	"shouldn't": "should not",
	"shouldn't've": "should not have",
	"so've": "so have",
	"so's": "so as / so is",
	"that'd": "that would / that had",
	"that'd've": "that would have",
	"that's": "that has / that is",
	"there'd": "there had / there would",
	"there'd've": "there would have",
	"there's": "there has / there is",
	"they'd": "they had / they would",
	"they'd've": "they would have",
	"they'll": "they shall / they will",
	"they'll've": "they shall have / they will have",
	"they're": "they are",
	"they've": "they have",
	"to've": "to have",
	"wasn't": "was not",
	"we'd": "we had / we would",
	"we'd've": "we would have",
	"we'll": "we will",
	"we'll've": "we will have",
	"we're": "we are",
	"we've": "we have",
	"weren't": "were not",
	"what'll": "what shall / what will",
	"what'll've": "what shall have / what will have",
	"what're": "what are",
	"what's": "what has / what is",
	"what've": "what have",
	"when's": "when has / when is",
	"when've": "when have",
	"where'd": "where did",
	"where's": "where has / where is",
	"where've": "where have",
	"who'll": "who shall / who will",
	"who'll've": "who shall have / who will have",
	"who's": "who has / who is",
	"who've": "who have",
	"why's": "why has / why is",
	"why've": "why have",
	"will've": "will have",
	"won't": "will not",
	"won't've": "will not have",
	"would've": "would have",
	"wouldn't": "would not",
	"wouldn't've": "would not have",
	"y'all": "you all",
	"y'all'd": "you all would",
	"y'all'd've": "you all would have",
	"y'all're": "you all are",
	"y'all've": "you all have",
	"you'd": "you had / you would",
	"you'd've": "you would have",
	"you'll": "you shall / you will",
	"you'll've": "you shall have / you will have",
	"you're": "you are",
	"you've": "you have"
}

def contractionfunction(text):
    cleaned_text = []
    for word in text.split(' '):
        if word in contractions:
          cleaned_text.append(contractions[word])
        else:
            cleaned_text.append(word)
    return cleaned_text

stop_words = set(stopwords.words('english'))
def remove_stopword(s):
    filtered_sentence = [w for w in s if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)

def data_preprocessing(text):
    text = comment_clean(text)
    text = remove_extra_space(text)
    text = remove_accented_chars(text)
    text = contractionfunction(text)
    text = remove_stopword(text)
    return text

In [ ]:
tqdm.pandas()
comment_train['comment_text'] = comment_train['comment_text'].str.lower().progress_apply(data_preprocessing)

  0%|          | 0/252279 [00:00<?, ?it/s]

In [ ]:
comment_test['comment_text'] = comment_test['comment_text'].str.lower().progress_apply(data_preprocessing)

  0%|          | 0/153164 [00:00<?, ?it/s]

In [ ]:
# generate after-preprocessed dataset
comment_train.to_csv('train.csv')
comment_test.to_csv('test.csv')

# Load clean data



In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/544 project/data/train.csv'
TEST_PATH = '/content/drive/MyDrive/544 project/data/test.csv'
comment_train = pd.read_csv(TRAIN_PATH)
comment_test = pd.read_csv(TEST_PATH)

In [ ]:
comment_train['toxic'].value_counts()

0    209226
1     43053
Name: toxic, dtype: int64

In [ ]:
commen

# Construct Model

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

In [ ]:
BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [ ]:
BATCH_SIZE = 64
final_data = shuffle(final_data, random_state=None)
train_data = final_data[int(len(final_data) * 0.3):].reset_index(drop=True)
val_data = train_data[int(len(train_data) * 0.8):].reset_index(drop=True)
train_data = train_data[:int(len(train_data) * 0.8)].reset_index(drop=True)
test_data = final_data[:int(len(final_data) * 0.3)].reset_index(drop=True)

class Data(Dataset):
    def __init__(self, tokenizer=BertTokenizer, train=0):
        if train == 0:
            self.data = train_data
        elif train == 1:
            self.data = test_data
        elif train == 2:
            self.data = val_data
        self.tokenizer = tokenizer
        self.X = []
        self.Y = []
        for i, (row) in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            x, y = self.row_to_tensor(self.tokenizer, row)
            self.X.append(x)
            self.Y.append(y)

    @staticmethod
    def row_to_tensor(tokenizer, row):
        tokens = tokenizer.encode(row['comment_text'], add_special_tokens=True)
        if len(tokens) > 120:
            tokens = tokens[:119] + [tokens[-1]]
        x = torch.LongTensor(tokens)
        y = torch.FloatTensor([row['toxic']])
        return x, y

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

def collate_fn(batch, device):
    x, y = list(zip(*batch))
    x = pad_sequence(x, batch_first=True, padding_value=0)
    y = torch.stack(y)
    return x.to(device), y.to(device)

trainSet = Data(tokenizer=tokenizer, train=0)
testSet = Data(tokenizer=tokenizer, train=1)
valSet = Data(tokenizer=tokenizer, train=2)
collate_fn = partial(collate_fn, device=device)

  0%|          | 0/141034 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (610 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/75554 [00:00<?, ?it/s]

  0%|          | 0/35259 [00:00<?, ?it/s]

In [ ]:
train_loader = DataLoader(
        trainSet,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn
    )

test_loader = DataLoader(
        testSet,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn
    )

val_loader = DataLoader(
        valSet,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn
    )

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(bert.config.hidden_size, 1)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask
            )
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
        cls_output = torch.sigmoid(cls_output)
        return cls_output

model = BertClassifier(BertModel.from_pretrained(BERT_MODEL_NAME)).to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def train(model, iterator, optimizer, loss_fn, scheduler):
    model.train()
    total_loss = 0
    for x, y in tqdm(iterator):
        optimizer.zero_grad()
        mask = (x != 0).float()
        outputs = model(x, attention_mask=mask, labels=y)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    print(f"Train loss {total_loss / len(iterator)}")

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
{'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
{'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
EPOCH_NUM = 2
warmup_steps = 10 ** 3
total_steps = len(train_loader) * EPOCH_NUM - warmup_steps
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

In [ ]:
loss_fn = nn.BCELoss()
for i in range(EPOCH_NUM):
    print('=' * 50, f"EPOCH {i}", '=' * 50)
    train(model, train_loader, optimizer, loss_fn, scheduler)
    break